## Load parameters and calculate quiescent luminosity, assign spectral class and  if not already

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import warnings
import specmatchemp.library
import specmatchemp.plots as smplot
import glob
warnings.filterwarnings("ignore")

from f_6_7_8 import merged_spec_class, Mbol_to_Lum, kepler_luminosity, L_quieterr

cluster_dict = {'M67':['M67','M67', 15, 4.0, 908.],
                'M44':['M44','M44', 47, 0.73, 160.],
                'Pleiades':['Pleiades','Pleiades',0.,0.100,135.]
               }
DM = lambda x: 5.*np.log10(float(x))-5. 

#------------------------------------------------
cluster = 'M67'
test = 'run_06'
l = 'stars_shortlist/{0}/results/{1}/catalog/{0}_parameter.csv'.format(cluster, test)
lback = 'stars_shortlist/{0}/results/{1}/catalog/{0}_parameter_backup.csv'.format(cluster, test)
#------------------------------------------------

/usr/lib64/python3.4/importlib/_bootstrap.py:321: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return f(*args, **kwds)


In [2]:
basics = cluster_dict[cluster]
dm = DM(basics[4])
global dm

print(glob.glob(l))

# if glob.glob(l) != []:
#     specs = pd.read_csv(lback, index_col='EPIC')
   
specs = pd.read_csv(l, index_col='EPIC')
# if 'L_quiet_bol_err' not in specs.columns.values:
wavmin, wavmax = 3480., 9700.
lib = specmatchemp.library.read_hdf(wavlim=[wavmin,wavmax])
#print(lib.library_params.columns.values)
specs = merged_spec_class(specs)

L_quiet, L_quiet_bol, L_quiet_err, L_quiet_bol_err = [],[],[],[], 
todrop1, whydrop1 = [], [] 

i=0
for oid in specs.index.values:
    print(i,oid)
    i+=1
    L_quiet_bol.append(Mbol_to_Lum(specs.Mbol[oid]))
    L_quiet_bol_err.append(Mbol_to_Lum(specs.Mbol[oid],specs.Mbol_err[oid],err=True,))
    Lq = kepler_luminosity(cluster,specs['T'][oid],specs.R_Rsun[oid],lib)
    
    if str(Lq) == 'nan':
        todrop1.append(True)
        whydrop1.append('noLum')
        L_quiet_err.append(np.nan)
    else:
        todrop1.append(False)
        whydrop1.append(None)
        L_quiet_err.append(L_quieterr(cluster,Lq,
                                    specs.R_Rsun[oid],
                                    specs.R_Rsun_err[oid],
                                    specs['T'][oid],
                                    specs['T_err'][oid],
                                    lib,))
    L_quiet.append(Lq)

specs['L_quiet'] = L_quiet
specs['L_quiet_bol'] = L_quiet_bol
specs['L_quiet_err'] = L_quiet_err
specs['L_quiet_bol_err'] = L_quiet_bol_err
specs['todrop1'] = todrop1
specs['whydrop1'] = whydrop1

['stars_shortlist/M67/results/run_06/catalog/M67_parameter.csv']
Reading library from /home/eilin/.specmatchemp/library.h5
67.0 3.605551275463989
56.625 5.829611908180509
50.375 7.548799573442125
64.5 4.444097208657794
69.75 8.212033852828421
51.875 4.013648589500581
43.875 9.319837713179345
51.125 6.112231589198825
53.5 11.20267825120404
69.375 6.223694642252301
65.875 3.6206870894900596
72.625 5.8081300777444715
62.75 4.710360920354193
49.375 5.764926278800103
51.25 6.53356717268599
51.5 8.276472678623424
70.125 7.235977819203152
56.0 3.122498999199199
56.125 18.360538526960475
70.125 5.946374946133148
65.375 5.498579361980693
65.25 4.575751304430781
51.0 7.664854858377946
69.0 6.383572667401852
53.75 6.475916923494309
59.875 1.4523687548277813
66.25 4.993746088859544
49.5 5.361902647381804
58.375 1.3169567191065923
55.5 19.1637678967368
66.875 6.881814804250402
52.0 8.0
47.875 8.53577032258952
69.125 7.355907489902249
55.375 3.6033838263498934
62.875 18.53670345557699
64.375 16.9995

5780.0 5980.0 0.9200000000000002 1.32 -0.22 0.28
15 211392354
5240.0 5440.0 0.6759999999999999 1.0759999999999998 -0.22 0.28
5240.0 5440.0 0.6759999999999999 1.0759999999999998 -0.22 0.28
16 211392455
3550.0 3750.0 0.26 0.66 -0.22 0.28
3550.0 3750.0 0.26 0.66 -0.22 0.28
17 211392789
5390.0 5590.0 0.7090000000000001 1.109 -0.22 0.28
5390.0 5590.0 0.7090000000000001 1.109 -0.22 0.28
18 211392837
4520.0 4720.0 0.48900000000000005 0.889 -0.22 0.28
4520.0 4720.0 0.48900000000000005 0.889 -0.22 0.28
19 211392859
3550.0 3750.0 0.26 0.66 -0.22 0.28
3550.0 3750.0 0.26 0.66 -0.22 0.28
20 211392899
3970.0 4170.0 0.454 0.8540000000000001 -0.22 0.28
3970.0 4170.0 0.454 0.8540000000000001 -0.22 0.28
21 211393067
3970.0 4170.0 0.454 0.8540000000000001 -0.22 0.28
3970.0 4170.0 0.454 0.8540000000000001 -0.22 0.28
22 211393129
5490.0 5690.0 0.7390000000000001 1.139 -0.22 0.28
5490.0 5690.0 0.7390000000000001 1.139 -0.22 0.28
23 211393422
3700.0 3900.0 0.335 0.7350000000000001 -0.22 0.28
3700.0 3900.0 0.

88 211400215
5940.0 6140.0 1.03 1.43 -0.22 0.28
5940.0 6140.0 1.03 1.43 -0.22 0.28
89 211400290
4520.0 4720.0 0.48900000000000005 0.889 -0.22 0.28
4520.0 4720.0 0.48900000000000005 0.889 -0.22 0.28
90 211400302
5620.0 5820.0 0.81 1.21 -0.22 0.28
5620.0 5820.0 0.81 1.21 -0.22 0.28
91 211400500
5490.0 5690.0 0.7390000000000001 1.139 -0.22 0.28
5490.0 5690.0 0.7390000000000001 1.139 -0.22 0.28
92 211400563
5820.0 6020.0 0.9200000000000002 1.32 -0.22 0.28
5820.0 6020.0 0.9200000000000002 1.32 -0.22 0.28
93 211400616
5940.0 6140.0 1.03 1.43 -0.22 0.28
5940.0 6140.0 1.03 1.43 -0.22 0.28
94 211400620
3550.0 3750.0 0.26 0.66 -0.22 0.28
3550.0 3750.0 0.26 0.66 -0.22 0.28
95 211400662
4130.0 4330.0 0.4609999999999999 0.861 -0.22 0.28
4130.0 4330.0 0.4609999999999999 0.861 -0.22 0.28
96 211400944
5390.0 5590.0 0.7090000000000001 1.109 -0.22 0.28
5390.0 5590.0 0.7090000000000001 1.109 -0.22 0.28
97 211400999
3600.0 3800.0 0.296 0.696 -0.22 0.28
3600.0 3800.0 0.296 0.696 -0.22 0.28
98 211401055
473

5070.0 5270.0 0.6140000000000001 1.014 -0.22 0.28
163 211412010
3550.0 3750.0 0.26 0.66 -0.22 0.28
3550.0 3750.0 0.26 0.66 -0.22 0.28
164 211412077
5490.0 5690.0 0.7390000000000001 1.139 -0.22 0.28
5490.0 5690.0 0.7390000000000001 1.139 -0.22 0.28
165 211412192
5560.0 5760.0 0.782 1.182 -0.22 0.28
5560.0 5760.0 0.782 1.182 -0.22 0.28
166 211412252
5780.0 5980.0 0.9200000000000002 1.32 -0.22 0.28
5780.0 5980.0 0.9200000000000002 1.32 -0.22 0.28
167 211412571
3970.0 4170.0 0.454 0.8540000000000001 -0.22 0.28
3970.0 4170.0 0.454 0.8540000000000001 -0.22 0.28
168 211412877
5670.0 5870.0 0.81 1.21 -0.22 0.28
5670.0 5870.0 0.81 1.21 -0.22 0.28
169 211413212
5580.0 5780.0 0.786 1.186 -0.22 0.28
5580.0 5780.0 0.786 1.186 -0.22 0.28
170 211413327
3770.0 3970.0 0.35899999999999993 0.7589999999999999 -0.22 0.28
3770.0 3970.0 0.35899999999999993 0.7589999999999999 -0.22 0.28
171 211413537
5670.0 5870.0 0.81 1.21 -0.22 0.28
5670.0 5870.0 0.81 1.21 -0.22 0.28
172 211413815
5670.0 5870.0 0.81 1.21 -0

5240.0 5440.0 0.6759999999999999 1.0759999999999998 -0.22 0.28
237 211427009
4520.0 4720.0 0.48900000000000005 0.889 -0.22 0.28
4520.0 4720.0 0.48900000000000005 0.889 -0.22 0.28
238 211427165
4310.0 4510.0 0.49799999999999994 0.8979999999999999 -0.22 0.28
4310.0 4510.0 0.49799999999999994 0.8979999999999999 -0.22 0.28
239 211427268
5620.0 5820.0 0.81 1.21 -0.22 0.28
5620.0 5820.0 0.81 1.21 -0.22 0.28
240 211427425
6070.0 6270.0 1.05 1.45 -0.22 0.28
6070.0 6270.0 1.05 1.45 -0.22 0.28
241 211427431
3900.0 4100.0 0.38700000000000007 0.7870000000000001 -0.22 0.28
3900.0 4100.0 0.38700000000000007 0.7870000000000001 -0.22 0.28
242 211427574
6070.0 6270.0 1.05 1.45 -0.22 0.28
6070.0 6270.0 1.05 1.45 -0.22 0.28
243 211427666
3970.0 4170.0 0.454 0.8540000000000001 -0.22 0.28
3970.0 4170.0 0.454 0.8540000000000001 -0.22 0.28
244 211427749
5390.0 5590.0 0.7090000000000001 1.109 -0.22 0.28
5390.0 5590.0 0.7090000000000001 1.109 -0.22 0.28
245 211427761
3000.0 3100.0 0.0 0.349 -0.22 0.28
3000.0 3

## All sorts of sanity checks

In [ ]:
specs[['g_r','J-H','H-K','r_i','i_z','z_J','J_H','H_K','R_Rsun','R_Rsun_err',
       'T','T_err','L_quiet','L_quiet_err','L_quiet_bol','L_quiet_bol_err','todrop1','whydrop1']].tail(20)

In [ ]:
series = pd.DataFrame({'color counts' : specs[['g_r','J-H','H-K','r_i','i_z','z_J','J_H','H_K','z-Y']].count()})
#series.to_latex()
series

In [ ]:
r = specs.L_quiet_err/specs.L_quiet
r.head()

In [ ]:
specs['todrop1'].value_counts().plot(kind='bar')

In [ ]:
specs.to_csv(l)


## CMD outlier detection

In [4]:
specs = pd.read_csv(l)
specs = specs.rename(index=str, columns={'e_gmag':'eSDSS_g','e_rmag':'eSDSS_r',
                                         'e_imag':'eSDSS_i','e_zmag':'eSDSS_z',
                                         'e_ymag':'eSDSS_y','J Mag Err':'eJ',
                                         'H Mag Err':'eH','K Mag Err':'eK',
                                        })

In [6]:
specs[['spec_class','EPIC']]

,spec_class,EPIC
0,M0,211389428
1,K2,211389537
2,G8,211389716
3,K8,211389931
4,M0.5,211390071
5,G8,211390107
6,G0,211390158
7,G8,211390800
8,K2,211391125
9,M1.5,211391208


In [ ]:

from f_6_7_8 import interactive_CMD

colors = [('g_r','SDSS_g','SDSS_r'),
          ('r_i','SDSS_r','SDSS_i'),
          #('i_z','SDSS_i','SDSS_z'),
          ('z_J','SDSS_z','J'),
          ('J_H','J','H'),
          ('H_K','H','K'),
          #('z-Y','SDSS_z','SDSS_y'),
          ('J-H','J','H'),
          ('H-K','H','K'),]

for key, c0, c1 in colors:
    print(c0,c1)
    interactive_CMD(specs,cid1=c0,cid2=c1)
    input('Plot for {} done?'.format(key))

In [ ]:
outs=pd.DataFrame()
import os
from f_6_7_8 import CMD
for i in range(len(colors)):
    #/home/ekaterina/Documents/appaloosa/stars_shortlist/share/temp/
    file = 'selected-data{}.txt'.format(i)
    if os.stat(file).st_size != 0:
        df = pd.read_csv(file,header=None,sep=' ').T.dropna(how='any')
        df = df.rename(index=str, columns={0:'outlier'})
        df['color'] = colors[i][0]
      #  print(specs.EPIC[df.outlier.astype(int).values].values)
        df['EPIC'] = specs.EPIC[df.outlier.astype(int).values].values
        outs = pd.concat([outs,df],ignore_index=True)

outs = outs.sort_values(by='EPIC')

#specs['todrop'][specs.EPIC.isin(outs.EPIC.tolist())] = True 
td = specs.todrop1.values
whydrop2 = np.chararray(td.shape,itemsize=3,unicode=True)

whydrop2[specs.EPIC.isin(outs.EPIC.tolist()).values] = 'CMD'
whydrop2[~specs.EPIC.isin(outs.EPIC.tolist()).values] = '   '
specs['whydrop2'] = whydrop2
specs['todrop2'] = specs.EPIC.isin(outs.EPIC.tolist()).values

outs.to_csv('stars_shortlist/{0}/results/{1}/catalog/{0}_outliers.csv'.format(cluster,test))
outs = outs.drop_duplicates()

specs['todrop'] = specs.todrop1 | specs.todrop2
specs['whydrop'] = specs.whydrop2 + ' ' +specs.whydrop1.astype(str).apply(lambda x: x.replace('nan','')) 
specs = specs.drop(['todrop1','todrop2','whydrop1','whydrop2'],axis=1)
#specs.whydrop[specs.todrop == True]

In [ ]:
specs['todrop'].value_counts().plot(kind='bar')

## Other immediate exclusion criteria: too hot/cold, no data

In [ ]:
specs.todrop[specs['T'] < 3000] = True
specs.todrop[specs['T'] > 6640] = True
specs.todrop[specs['T'] == np.nan] = True
specs.todrop[specs['L_quiet'] == np.nan] = True

specs.whydrop[specs['T'] < 3000] = 'T < 3000 K'
specs.whydrop[specs['T'] > 6640] = 'earlier than F4'
specs.whydrop[specs['T'] == np.nan] = 'no T'
specs.whydrop[specs['L_quiet'] == np.nan] = 'no L'

#intermediate save
specs.to_csv(l)

In [ ]:
specs[specs.todrop!=False]

In [ ]:
specs.whydrop.value_counts().plot(kind='bar')

## CMD plots

In [ ]:
from pandas.api.types import CategoricalDtype
from f_6_7_8 import spec_class_hist
categor = ['{}{}'.format(c,num) for c in list('OBAFGKM') for num in list(range(10))]
type_ = CategoricalDtype(categories=categor, ordered=True)
specs.spec_class.astype(type_)
s = specs.sort_values(by='spec_class')
spec_class_hist(s,cluster,type_)

In [ ]:
from f_6_7_8 import CMD
outliers = outs.EPIC.drop_duplicates().tolist()
for key, c0, c1 in colors:
    CMD(specs,cluster,cid1=c0,cid2=c1,colour=key, ylim=(18,9),outliers=outliers)#,colour=key

In [ ]:
CMD(specs,cluster,cid1='gmag',cid2='imag',color='g_i',ylim=(21,8))#19,5
CMD(specs,cluster,cid1='J',cid2='K',color='J_K',ylim=(18,6))#14,5
CMD(specs,cluster,cid1='rmag',cid2='zmag',color='r_z',ylim=(20,8))#18,5
CMD(specs,cluster,cid1='gmag',cid2='zmag',color='g_z',ylim=(20,9))#18,5